In [1]:
import asyncio
import aiohttp
from ollama import Client

# Use your local Ollama instance
client = Client(host='http://host.docker.internal:11434')

# List of pages to process
pages = [
    "/workspaces/smart-library/data/pdf/pages/BrouwerSimm2019/p02.txt",
    "/workspaces/smart-library/data/pdf/pages/ChenZhou2025/p07.txt",
    "/workspaces/smart-library/data/pdf/pages/DuDai2016/p03.txt",
    "/workspaces/smart-library/data/pdf/pages/YoonZame2018/p07.txt"
]

# ----------------------------------------
# Safe concurrency settings for 8 GB VRAM
# ----------------------------------------
MAX_CONCURRENCY = 1     # 1 = fully safe, 2 = maybe ok, 3 = unsafe
sem = asyncio.Semaphore(MAX_CONCURRENCY)


async def run_ollama(prompt):
    """Run one request safely with concurrency limit."""
    async with sem:
        # We wrap client.generate in a thread executor
        # because Ollama's Python client is sync.
        loop = asyncio.get_event_loop()
        return await loop.run_in_executor(
            None,
            lambda: client.generate(
                model="llama3",
                prompt=prompt,
                format="json",
                options={"temperature": 0}
            )
        )


async def process_page(path):
    """Load a page, generate the prompt, call Ollama."""
    text = open(path, "r").read()
    prompt = get_prompt(text)
    response = await run_ollama(prompt)
    print(f"\n=== RESULTS FOR {path} ===")
    print(response["response"])
    return response


async def main():
    tasks = [process_page(p) for p in pages]
    await asyncio.gather(*tasks)


# Run it
asyncio.run(main())


ModuleNotFoundError: No module named 'aiohttp'